In [ ]:
from pathlib import Path
import numpy as np
import auxiliary_functions as aux 

In [ ]:

fname_dict = Path("/media/sf_CCPPETMR/TestData/Input/xDynamicSimulation/pDynamicSimulation/Fingerprints/dict_70_1500.npz")
mrfdict = np.load(fname_dict)

dict_theta = mrfdict['dict_theta']

dict_mrf = mrfdict['dict_norm']
dict_mrf = np.transpose( aux.apply_databased_sliding_window(ad, np.transpose(dict_mrf)))

downscaling_factor = 10
reduced_dict_mrf = dict_mrf[0:-1:downscaling_factor, ...]
reduced_dict_theta = dict_theta[0:-1:downscaling_factor, ...]




In [ ]:

img_series = recon.as_array()
img_shape = img_series.shape[1:]
img_series_1d = np.transpose(np.reshape(img_series,(img_series.shape[0], -1)))
print(img_shape)


parametric_maps = aux.match_dict_1d(reduced_dict_mrf, reduced_dict_theta, img_series_1d)


dict_match = np.reshape(parametric_maps[0], (*img_shape, -1))

import matplotlib.pyplot as plt
f,ax = plt.subplots(1,3)
ax[0].imshow(dict_match[:,:,0])
ax[0].axis("off")
pcm = ax[1].imshow(dict_match[:,:,1])
ax[1].axis("off")
f.colorbar(pcm, ax=ax[1], shrink=0.3)
pcm = ax[2].imshow(dict_match[:,:,2])
f.colorbar(pcm, ax=ax[2], shrink=0.3)
ax[2].axis("off")

plt.show()

import nibabel as nib
img = nib.Nifti1Image(np.abs(dict_match[...,1]), np.eye(4))
nib.save(img,"/media/sf_CCPPETMR/TMP_T1_FIT.nii")

img = nib.Nifti1Image(np.abs(dict_match[...,2]), np.eye(4))
nib.save(img,"/media/sf_CCPPETMR/TMP_T2_FIT.nii")